# HW3_Q2  DSC 478

For this problem you will use a different subset of the 20 Newsgroup data set that you used in Assignment 2  (see the description of the full dataset). The subset for this assignment includes 2,500 documents (newsgroup posts), each belonging to one of 5 categories windows (0), crypt (1), christian (2), hockey (3), forsale (4). The documents are represented by 9328 terms (stems). The dictionary (vocabulary) for the data set is given in the file "terms.txt" and the full term-by-document matrix is given in "matrix.txt" (comma separated values). The actual category labels for the documents are provided in the file "classes.txt". Your goal in this assignment is to perform clustering on the documents and compare the clusters to the actual categories.

<b>Your tasks in this problem are the following Note: for the clustering part of this assignment you should use the kMeans module form Ch. 10 of MLA (use the version provided here as it includes some corrections to the book version). You may also use Pandas and other modules from scikit-learn that you may need for preprocessing or evaluation.]

In [160]:
# Importing modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import os 
import math

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn import feature_selection
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectPercentile, f_regression

<b> A. Create your own distance function that, instead of using Euclidean distance, uses Cosine similarity. This is the distance function you will use to pass to the kMeans function.

In [161]:
# working directory chosen
os.chdir('/Users/burcu.alagoz/Desktop/HW3/newsgroups5')

In [162]:
os.getcwd()

'/Users/burcu.alagoz/Desktop/HW3/newsgroups5'

In [172]:
#function
#vectors
def cosSim(vecA, vecB):
    norm_A = linalg.norm(vecA)
    norm_B = linalg.norm(vecB)
# manually computing cosine similarity
    cosine = dot(vecA, vecB) / (norm_A * norm_B)
    distance = 1 - cosine
    return distance


<b>B.Load the data set Note: the data matrix provided has terms as rows and documents as columns. Since you will be clustering documents, you'll need to take the transpose of this matrix so that your main data matrix is a document x term matrix. In Numpy, you may use the ".T" operation to obtain the transpose.] Then, split the data set (the document x term matrix) and set aside 20% for later use (see below). Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.

In [173]:
# working directory chosen
os.chdir('/Users/burcu.alagoz/Desktop/HW3/newsgroups5')

In [174]:
# Loading the data set
matrix = pd.DataFrame(np.genfromtxt("matrix.txt", delimiter=',',dtype=int))
matrix.head()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [175]:
matrixT=matrix.T

In [176]:
classes = np.genfromtxt("classes.txt", delimiter=' ', dtype=int, skip_header=1, usecols=(1))
classes.shape

(2500,)

In [177]:
classes

array([0, 1, 1, ..., 3, 4, 2])

In [178]:
terms = pd.read_table('terms.txt', sep='\n', header=None)
terms.shape

(9328, 1)

In [179]:
terms.head(10)

,0
0,aa
1,aargh
2,aaron
3,aaronc
4,ab
5,abandon
6,abc
7,abid
8,abil
9,abl


In [181]:
# split the data set (the document x term matrix) and set aside 20% for later use (see below). 
#Use the 80% segment for clustering in the next part. The 20% portion must be a random subset.
train, test, classes_train, classes_test = train_test_split(matrixT, classes, test_size=0.2, random_state=33)
print(train.shape, test.shape, classes_train.shape, classes_test.shape)

(2000, 9328) (500, 9328) (2000,) (500,)


In [182]:
#Final dataset
matrixT = matrix.T
matrixT.head()

,0,1,2,3,4,5,6,7,8,9,...,9318,9319,9320,9321,9322,9323,9324,9325,9326,9327
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [183]:
matrixT.shape

(2500, 9328)

In [184]:
term_set = pd.DataFrame(np.genfromtxt("terms.txt", delimiter='\t',dtype=str))
term_set.head()

,0
0,aa
1,aargh
2,aaron
3,aaronc
4,ab


In [185]:
#Final term_set
target_terms = np.array(term_set)
target_terms[:5,:]

array([['aa'],
       ['aargh'],
       ['aaron'],
       ['aaronc'],
       ['ab']], dtype=object)

In [186]:
#Final Class set
classes = pd.DataFrame(np.genfromtxt("classes.txt", delimiter=' ',dtype=int,skip_header=1))
classes = classes.drop(0, axis=1)

In [187]:
classes.head()

,1
0,0
1,1
2,1
3,1
4,2


In [188]:
#DF = number of documents the term appears - summing along the rows
df = np.array([(matrixT!=0).sum(0)])
print (df )

[[10  6 22 ...  3  3  4]]


In [189]:
n_docs = len(matrixT[0])
print (n_docs)

2500


In [190]:
#Create a matrix of ones and multiply by your docs
n_matrix = np.ones(np.shape(matrixT), dtype=float)*n_docs

In [191]:
divided_m = np.divide(n_matrix, df)
IDF = np.log2(divided_m)

In [192]:
np.set_printoptions(precision=2, suppress=True)
print (IDF[0:2,])

[[7.97 8.7  6.83 ... 9.7  9.7  9.29]
 [7.97 8.7  6.83 ... 9.7  9.7  9.29]]


In [193]:
np.set_printoptions(precision=2, suppress=True)
print (IDF[0:2,])

[[7.97 8.7  6.83 ... 9.7  9.7  9.29]
 [7.97 8.7  6.83 ... 9.7  9.7  9.29]]


In [210]:
#doc-term tfidf matrix= docxterm matrix *IDF 
docterm_tfidf = matrixT * IDF

<b>C.Perform Kmeans clustering on the training data. Write a function to display the top N terms in each cluster along with the cluster DF values for each term and the size of the cluster. The cluster DF value for a term t in a cluster C is the percentage of docs in cluster C in which term t appears (so, if a cluster has 500 documents, and term "game" appears in 100 of those 500 documents, then DF value of "game" in that cluster is 0.2 or 20%). Sort the terms for each cluster in decreasing order of the DF percentage. Here is an example of how this output might look like (here the top 10 terms for 3 of the 5 clusters are displayed in decreasing order of cluster DF values, but the mean frequency from the cluster centroid is also shown). [Extra Credit: use your favorite third party tool, ideally with a Python based API, to create a word cloud for each cluster.]

In [211]:
from numpy import *

#euclidean distance function
def distEuc(value1, value2):
    return sqrt(sum(power(value1 - value2, 2)))

def random_centroid(dataset, k):
    n = shape(dataset)[1]
    centroids = zeros((k,n), dtype = float)
    for j in range(n):
        j_min = min(dataset[:, j])
        j_range = float(max(dataset[:, j]) - j_min)
        centroids[:, j] = j_min + j_range * random.rand(k)    
    return centroids


def k_Means(dataset, k, dist_meas = cosSim, create_cent = random_centroid):
    m = shape(dataset)[0]
    centroids = create_cent(dataset, k)
    cluster_assgnment = zeros((m, 2))
    cluster_changed = True
    
    while  cluster_changed:
        cluster_changed = False
        for i in range(m): #assigning each data point to the closest centroid
            min_dist = np.inf; min_index = -1
            
            for j in range(k):
                dist_ji = dist_meas(centroids[j, :],dataset[i, :])
                if dist_ji < min_dist:
                    min_dist = dist_ji; min_index = j
        
            if dist_ji < min_dist:
                    min_dist = dist_ji; min_index = j
            if cluster_assgnment[i,0] != min_index: cluster_changed = True
            cluster_assgnment[i,:] = min_index,min_dist**2
            
        for c in range(k):  #printing centroids 
            points = dataset[nonzero(cluster_assgnment[:, 0] == c)[0]] #cluster points 
            if(len(points)!=0):
                centroids[c,:] = mean(points, axis=0)
    return centroids, cluster_assgnment  #assigning centroid to the Mean

In [218]:
def top_terms(data, K, num_terms):
    centroids, cluster_assgnment = k_Means(mat(data), K, dist_meas = cosSim)
    
    for i in range(K):
        cluster_matrix = data[cluster_assgnment[:, 0] == i]
        cluster_freq = np.array([(cluster_matrix.T != 0).sum(1)]).T
        print('CLUSTER', i+1, ': ')
        print('Number of docs in Cluster ',i+1 ,': ', cluster_matrix.shape[0])        
        
        matrix_c = cluster_freq / float(cluster_matrix.shape[0])
        matrix_c = map(list, matrix_c)
        matrix_c = [m[0] for m in matrix_c]
        cluster_freq = [m[0] for m in cluster_freq]
        n = sorted(zip(terms, cluster_freq, matrix_c),key = lambda x:x[2], reverse = True)
        for m in n[:num_terms]:
            print('Term: ',m[0])
            print('Frequency: ',m[1])
            print('Percent of docs contain term ', m[0], ': ', m[2] * 100,'\n')
    return centroids, cluster_assgnment


In [ ]:
centroids, cluster_assgnment = top_terms(dt_tfidf_train, 5,10)

<b>D.Using the cluster assignments from Kmeans clustering, compare your 5 clusters to the 5 pre-assigned classes by computing the Completeness and Homogeneity values.

In [224]:
from sklearn.metrics import completeness_score, homogeneity_score

In [225]:
cs = completeness_score(np.array(train.T)[0],cluster_assgnment[:,0])
print('Completeness score: %0.8f' % cs)

Completeness score: 0.00033594


In [226]:
hs = homogeneity_score(np.array(train.T)[0],cluster_assgnment[:,0])
print('Homogeneity of Clusters: %0.8f' % hs)

Homogeneity of Clusters: 0.01002326


<b>E.Finally, using your cluster assignments as class labels, categorize each of the documents in the 20% set-aside data into each of the appropriate cluster. Your categorization should be based on Cosine similarity between each test document and cluster centroids. For each test document show the predicted class label as well as Cosine similarity to the corresponding cluster.

In [ ]:
output = []
for d in dt_tfidf_test:  
    dic = {}
    sim = []
    c = 0
    for centroid in cent:
        c += 1
        cmatrix_sim = 1 - cosineSim(d, centroid)
        sim.append(cmatrix_sim) 
        dic['Cosine Similarity' + str(c)] = cmatrix_sim
    dic['Predicted Cluster'] = sim.index(max(sim)) + 1
    results.append(dic)
test_cluster = pd.DataFrame(results)